# Positioning Redundant Mannipulator Performance 

In [1]:
import robosandbox as rsb
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
import matplotlib.pyplot as plt
from robosandbox.performance.WorkSpace import WorkSpace
import plotly.express as px
import pandas as pd


## Definition 

The generic manipulator of 4 Dofs is defined in DH description and the actuators' directions from 2 to 4 and EE are defined by alpha list.

In [5]:
%matplotlib qt
robot = rsb.models.DH.Generic.GenericFour(alpha=[np.pi/2, 0, 0, 0])
robot.plot(q=robot.qr)
plt.savefig("fig/robot_redundant.png")

![redundant robot](fig/robot_redundant.png)

## Evaluate Alpha 3&4 Effects on Global Indices

In [6]:
def evaluate_alpha_effect(
    alpha_list3, alpha_list4, isSave=False, method="invcondition", axes="all"
):
    """
    Evaluate the effect of alpha on the global indices.
    """
    results = []
    res_mat = np.zeros((len(alpha_list3), len(alpha_list4)))
    for i, alpha3 in tqdm(
        enumerate(alpha_list3),
        total=len(alpha_list3),
        desc="alpha3 progress",
        unit="outer",
    ):
        for j, alpha4 in tqdm(
            enumerate(alpha_list4),
            total=len(alpha_list4),
            desc=f"alpha4 progress (alpha3={np.rad2deg(alpha3)})",
            unit="inner",
            leave=False,
        ):
            robot = rsb.models.DH.Generic.GenericFour(alpha=[np.pi / 2, alpha3, alpha4, 0])
            ws = WorkSpace(robot=robot)
            G = ws.iter_calc_global_indice(
                    initial_samples=3000,
                    batch_ratio=0.1,
                    error_tolerance_percentage=1e-3,
                    method=method,
                    axes=axes,
                    max_samples=30000,
                )
            results.append([alpha3, alpha4, G])
            res_mat[i, j] = G
            del robot, ws

    results = np.array(results)
    if isSave:
        filename = f"data/two_alpha/{method}_{axes}.npz"
        np.savez(filename, results=results, res_mat=res_mat)
    return results, res_mat


In [7]:
def plot_global_index_results(
    alpha_list_deg3,
    alpha_list_deg4,
    res_mat,
    plot_type="heatmap",
    method="invcondition",
    axes="all",
    isSave=False,
):
    """
    Plot the effect of alpha on the global indices using Plotly.
    """
    G_mat = res_mat
    # print(G_mat)
    fig = go.Figure()
    if plot_type == "heatmap":
        fig = go.Figure(
            data=go.Heatmap(
                z=G_mat,
                x=alpha_list_deg3,
                y=alpha_list_deg4,
                # colorscale='Viridis',
                colorbar=dict(
                    title=f"{method}", titlefont=dict(size=40), tickfont=dict(size=40)
                ),
            )
        )

    if plot_type == "surface":
        fig = go.Figure(
            data=go.Surface(
                z=G_mat,
                x=alpha_list_deg3,
                y=alpha_list_deg4,
                colorbar=dict(
                    title=f"{method}", titlefont=dict(size=40), tickfont=dict(size=40)
                ),
            )
        )
        fig.update_layout(
            scene=dict(
                xaxis_title="alpha3 (deg)",
                yaxis_title="alpha4 (deg)",
                zaxis_title="",
                xaxis=dict(titlefont=dict(size=40), tickfont=dict(size=16), dtick=90),
                yaxis=dict(titlefont=dict(size=40), tickfont=dict(size=16), dtick=90),
                zaxis=dict(titlefont=dict(size=40), tickfont=dict(size=16), dtick=90),
                camera=dict(
                    eye=dict(x=1.55, y=1.55, z=1.55),
                    up=dict(x=0, y=0, z=1),
                ),
            )
        )

    fontsize = 40
    fig.update_layout(
        # title=f'Effect of alpha on global indices using {method} method and {axes} axes',
        xaxis_title="alpha3 (deg)",
        yaxis_title="alpha4 (deg)",
        autosize=True,
        height=800,
        width=1000,
        xaxis_title_font=dict(size=40),  # Font size for x-axis title
        yaxis_title_font=dict(size=40),
        xaxis=dict(tickfont=dict(size=fontsize), dtick=30),
        yaxis=dict(tickfont=dict(size=fontsize), dtick=30),
    )

    if isSave:
        fig.write_image(f"fig/two_alpha/{method}_{axes}_{plot_type}.png")
        fig.write_html(f"fig/two_alpha/{method}_{axes}_{plot_type}.html")
    fig.show()

### Global Indice - Manipulability from Yoshikawa

$$
\sqrt{\det(J(q)J^T(q))}
$$

In [10]:
np.random.seed(42)

# from 0, 10, 20, ..., 180 deg for alpha
alpha_list_deg3 = np.arange(0, 181, 15)
alpha_list_rad3 = np.deg2rad(alpha_list_deg3)
alpha_list_deg4 = np.arange(0, 181, 15)
alpha_list_rad4 = np.deg2rad(alpha_list_deg4)
print(f"alpha 3: {alpha_list_deg3}")
print(f"alpha 4: {alpha_list_deg4}")

method = "yoshikawa"
axes = "all"

# isRun = True
# isSave = True

isSave = False
isRun = False

if isRun:
    res, res_mat = evaluate_alpha_effect(
        alpha_list_rad3, alpha_list_rad4, isSave=True, method=method, axes=axes
    )
if not isRun:
    npzfile = np.load(f"data/two_alpha/{method}_{axes}.npz")
    res = npzfile["results"]
    res_mat = npzfile["res_mat"]
plot_global_index_results(
    alpha_list_deg3,
    alpha_list_deg4,
    res_mat,
    plot_type="surface",
    method=method,
    axes=axes,
    isSave=isSave,
)
plot_global_index_results(
    alpha_list_deg3,
    alpha_list_deg4,
    res_mat,
    plot_type="heatmap",
    method=method,
    axes=axes,
    isSave=isSave,
)

alpha 3: [  0  15  30  45  60  75  90 105 120 135 150 165 180]
alpha 4: [  0  15  30  45  60  75  90 105 120 135 150 165 180]


### Global Indice - Inverse Condition Number
$$\sigma_{\text{min}}/\sigma_{\text{max}}$$

In [12]:
np.random.seed(42)

# from 0, 10, 20, ..., 180 deg for alpha
alpha_list_deg3 = np.arange(0, 181, 15)
alpha_list_rad3 = np.deg2rad(alpha_list_deg3)
alpha_list_deg4 = np.arange(0, 181, 15)
alpha_list_rad4 = np.deg2rad(alpha_list_deg4)
print(f"alpha 3: {alpha_list_deg3}")
print(f"alpha 4: {alpha_list_deg4}")

method = "invcondition"
axes = "all"

# isRun = True
# isSave = True

isSave = False
isRun = False

if isRun:
    res, res_mat = evaluate_alpha_effect(
        alpha_list_rad3, alpha_list_rad4, isSave=True, method=method, axes=axes
    )
if not isRun:
    npzfile = np.load(f"data/two_alpha/{method}_{axes}.npz")
    res = npzfile["results"]
    res_mat = npzfile["res_mat"]


plot_global_index_results(
    alpha_list_deg3,
    alpha_list_deg4,
    res_mat,
    plot_type="surface",
    method=method,
    axes=axes,
    isSave=isSave,
)

plot_global_index_results(
    alpha_list_deg3,
    alpha_list_deg4,
    res_mat,
    plot_type="heatmap",
    method=method,
    axes=axes,
    isSave=isSave,
)

alpha 3: [  0  15  30  45  60  75  90 105 120 135 150 165 180]
alpha 4: [  0  15  30  45  60  75  90 105 120 135 150 165 180]


### Global Indice - Minimum Singular Value
$$
\sigma_{\text{min}}
$$

In [14]:
np.random.seed(42)

# from 0, 10, 20, ..., 180 deg for alpha
alpha_list_deg3 = np.arange(0, 181, 15)
alpha_list_rad3 = np.deg2rad(alpha_list_deg3)
alpha_list_deg4 = np.arange(0, 181, 15)
alpha_list_rad4 = np.deg2rad(alpha_list_deg4)
print(f"alpha 3: {alpha_list_deg3}")
print(f"alpha 4: {alpha_list_deg4}")

method = "minsingular"
axes = "all"

# isRun = True
# isSave = True

isSave = False
isRun = False

if isRun:
    res, res_mat = evaluate_alpha_effect(
        alpha_list_rad3, alpha_list_rad4, isSave=True, method=method, axes=axes
    )
if not isRun:
    npzfile = np.load(f"data/two_alpha/{method}_{axes}.npz")
    res = npzfile["results"]
    res_mat = npzfile["res_mat"]


plot_global_index_results(
    alpha_list_deg3,
    alpha_list_deg4,
    res_mat,
    plot_type="surface",
    method=method,
    axes=axes,
    isSave=isSave,
)

plot_global_index_results(
    alpha_list_deg3,
    alpha_list_deg4,
    res_mat,
    plot_type="heatmap",
    method=method,
    axes=axes,
    isSave=isSave,
)

alpha 3: [  0  15  30  45  60  75  90 105 120 135 150 165 180]
alpha 4: [  0  15  30  45  60  75  90 105 120 135 150 165 180]


## Evaluate Alpha 2&3&4 Effects on Global Indices

In [15]:
def evaluate_3alpha_effect(
    alpha_list2, alpha_list3, alpha_list4, isSave=False, method="invcondition", axes="all"
):
    """
    Evaluate the effect of alpha on the global indices.
    """
    results = []
    res_mat = np.zeros((len(alpha_list2), len(alpha_list3), len(alpha_list4)))
    for i, alpha2 in tqdm(
        enumerate(alpha_list2),
        total=len(alpha_list2),
        desc="alpha2 progress",
        unit="outer",
    ):
        for j, alpha3 in tqdm(
            enumerate(alpha_list3),
            total=len(alpha_list3),
            desc=f"alpha3 progress (alpha2={np.rad2deg(alpha2)})",
            unit="inner",
            leave=False,
        ):
            for k, alpha4 in tqdm(
                enumerate(alpha_list4),
                total=len(alpha_list4),
                desc=f"alpha4 progress (alpha2={np.rad2deg(alpha2)}, alpha3={np.rad2deg(alpha3)})",
                unit="inner",
                leave=False,
            ):
                robot = rsb.models.DH.Generic.GenericFour(alpha=[alpha2, alpha3, alpha4, 0])
                ws = WorkSpace(robot=robot)
                G = ws.iter_calc_global_indice(
                    initial_samples=3000,
                    batch_ratio=0.1,
                    error_tolerance_percentage=1e-3,
                    method=method,
                    axes=axes,
                    max_samples=30000,
                )
                results.append([alpha2, alpha3, alpha4, G])
                res_mat[i, j, k] = G
                del robot, ws

    results = np.array(results)
    if isSave:
        filename = f"data/three_alpha/{method}_{axes}.npz"
        np.savez(filename, results=results, res_mat=res_mat)
    return results, res_mat


def plot_scatter_global_index_results(
    alpha_list_deg2,
    alpha_list_deg3,
    alpha_list_deg4,
    res,
    res_mat,
    plot_type="scatter",
    method="invcondition",
    axes="all",
    isSave=False,
):
    G_mat= res[:,-1].copy()
    if plot_type == "scatter":
        fig = go.Figure(
            data=go.Scatter3d(
                x=np.rad2deg(res[:, 0]),
                y=np.rad2deg(res[:, 1]),
                z=np.rad2deg(res[:, 2]),
                mode="markers",
                marker=dict(
                    size=10,
                    color=G_mat,
                    # colorscale="Viridis",
                    colorbar=dict(title=f"{method}", titlefont=dict(size=40), tickfont=dict(size=40)),
                ),
            )
        )
        fig.update_layout(
            scene=dict(
                xaxis_title="alpha2 (deg)",
                yaxis_title="alpha3 (deg)",
                zaxis_title="alpha4 (deg)",
                xaxis=dict(titlefont=dict(size=40), tickfont=dict(size=16), dtick=90),
                yaxis=dict(titlefont=dict(size=40), tickfont=dict(size=16), dtick=90),
                zaxis=dict(titlefont=dict(size=40), tickfont=dict(size=16), dtick=90),
                camera=dict(
                    eye=dict(x=1.55, y=1.55, z=1.55),
                    up=dict(x=0, y=0, z=1),
                ),
            )
        )

    if plot_type == 'parallel': 
        # Convert the data to a DataFrame
        columns = ['alpha2', 'alpha3', 'alpha4', 'G']
        res[:,:3] = np.rad2deg(res[:,:3])
        df = pd.DataFrame(res, columns=columns)
        fig = px.parallel_coordinates(
            df, color='G',
            labels={'0': 'alpha2 (deg)', '1': 'alpha3 (deg)', '2': 'alpha4 (deg)'},
            # color_continuous_scale='Viridis',
            height=800,
            width=1000,
        )
    if plot_type == "surface":
        print("PLotting Surface")
        fig = go.Figure(
            data=go.Surface(
                x=res[:, 0],
                y=res[:, 1],
                z=res[:, 2],
                surfacecolor=res_mat,
            )
        )
        fig.update_layout(
            scene=dict(
                xaxis_title="alpha2 (deg)",
                yaxis_title="alpha3 (deg)",
                zaxis_title="alpha4 (deg)",
                xaxis=dict(titlefont=dict(size=40), tickfont=dict(size=16), dtick=90),
                yaxis=dict(titlefont=dict(size=40), tickfont=dict(size=16), dtick=90),
                zaxis=dict(titlefont=dict(size=40), tickfont=dict(size=16), dtick=90),
                camera=dict(
                    eye=dict(x=1.55, y=1.55, z=1.55),
                    up=dict(x=0, y=0, z=1),
                ),
            )
        )
        
    # df = px.data.iris()
    # fig = px.scatter_3d(
    #     x=res[:, 0],
    #     y=res[:, 1],
    #     z=res[:, 2],
    #     color=G_mat,
    #     title=f'Effect of alpha on global indices using {method} method and {axes} axes',
    #     labels={'x': 'alpha2 (deg)', 'y': 'alpha3 (deg)', 'z': 'alpha4 (deg)'},
    #     color_continuous_scale='Viridis',
    #     height=800,
    #     width=1000,
    # )
    if isSave:
        fig.write_image(f"fig/three_alpha/{method}_{axes}_{plot_type}.png")
        fig.write_html(f"fig/three_alpha/{method}_{axes}_{plot_type}.html")
    fig.show()

    

### Global Indice - Inverse Condition Number

In [ ]:
np.random.seed(42)

# from 0, 10, 20, ..., 180 deg for alpha
alpha_list_deg2 = np.arange(-90, 91, 15)
alpha_list_rad2 = np.deg2rad(alpha_list_deg2)
alpha_list_deg3 = np.arange(-90, 91, 15)
alpha_list_rad3 = np.deg2rad(alpha_list_deg3)
alpha_list_deg4 = np.arange(-90, 91, 15)
alpha_list_rad4 = np.deg2rad(alpha_list_deg4)
print(f"alpha 2: {alpha_list_deg2}")
print(f"alpha 3: {alpha_list_deg3}")
print(f"alpha 4: {alpha_list_deg4}")

method = "invcondition"
axes = "all"

isRun = True
isSave = True

# isSave = False
# isRun = False

if isRun:
    res, res_mat = evaluate_3alpha_effect(
        alpha_list_rad2, alpha_list_rad3, alpha_list_rad3, isSave=True, method=method, axes=axes
    )
if not isRun:
    npzfile = np.load(f"data/three_alpha/{method}_{axes}.npz")
    res = npzfile["results"]
    res_mat = npzfile["res_mat"]

plot_scatter_global_index_results(
    alpha_list_deg2,
    alpha_list_deg3,
    alpha_list_deg4,
    res,
    res_mat,
    plot_type="parallel",
    method=method,
    axes=axes,
    isSave=isSave,
)


alpha 2: [ 0 15 30 45 60 75 90]
alpha 3: [ 0 15 30 45 60 75 90]
alpha 4: [ 0 15 30 45 60 75 90]


In [21]:
robot_g = rsb.models.DH.Generic.GenericFour(alpha=[np.deg2rad(-90), 0, np.deg2rad(30), 0])
robot_g.plot(q=robot_g.qz)
robot_g.manipulability(q=[0.1, 0.2, 0.3, 0.4], method='minsingular')

0.12535610491431737